In [3]:
import numpy as np
import math
import copy
from   more_itertools import powerset
from operator import itemgetter
#import pandas as pd
from   ucimlrepo import fetch_ucirepo
from   sklearn.model_selection import train_test_split


In [4]:
# Load and split the data
wine_quality = fetch_ucirepo(id=186)
wine_subset  = wine_quality['data']['original'][wine_quality['data']['original']['color'] == 'white']
X = np.array(wine_subset[['fixed_acidity', 'volatile_acidity', 'citric_acid','residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol']])
y = np.array(wine_subset['quality'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
player_names = ['fixed_acidity', 'volatile_acidity', 'citric_acid','residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol']
N = set([i for i in range(X.shape[1])])

In [6]:
# Standardise the training data
X_mu, X_sigma, y_mu, y_sigma = X_train.mean(axis=0), X_train.std(axis=0), y_train.mean(axis=0), y_train.std(axis=0)

X_train = (X_train - X_mu) / X_sigma
y_train = (y_train - y_mu) / y_sigma

# Standardise the test data
X_test = (X_test - X_mu)/X_sigma

$$X^TXw = X^Ty$$ 
$$w=(X^TX)^{-1}X^Ty$$

In [7]:
w = np.matmul(np.matmul(np.linalg.inv(np.matmul(X_train.T, X_train)), X_train.T), y_train)

$$R^2  = \frac{1}{N}w^TX^TXw$$
$$R^2_i = \frac{1}{N}w_i (X^TXw)$$

In [8]:
R2  = np.matmul(w.T,np.matmul(np.matmul(X_train.T, X_train), w))/len(y_train)
R2i = np.matmul(np.diag(w),np.matmul(np.matmul(X_train.T, X_train), w))/len(y_train)
print(R2)
print(R2i) 

0.2843126626414035
[-5.15764443e-03  4.48079867e-02  1.16066061e-04 -3.99818409e-02
  1.29593436e-04  2.77336079e-03  1.86662385e-03  1.27245440e-01
  1.04933860e-02  5.15467153e-03  1.36865019e-01]


$$\hat{y} = Xw$$
$$MSE = \frac{1}{N}\sum(y-\hat{y})^2$$

In [9]:
yh  = np.matmul(X_test, w)
MSE = np.matmul((y_test - yh).T, (y_test - yh)) / len(y_test)

In [10]:
w

array([ 0.04371106, -0.2193717 , -0.00827766,  0.41280525, -0.00065067,
        0.09626305, -0.01148575, -0.42336479,  0.10171807,  0.08312901,
        0.3170512 ])

Three approaches to sampling have been implemented:

1.  sample_v1 samples whole vectors
2.  sample v2 samples individual fields in turn to produce a set of vectors
3.  sample v3 exploits the fact that $\mathbb{E}[f(x)] = f(\mathbb{E}[x])$ for linear models

In [11]:
def predict(X_vector):
    y_standard = np.matmul(X_vector, w)
    y_actual   = (y_standard * y_sigma) + y_mu
    return y_actual

#E_f = np.mean(predict(X_test)) # should be same as sampling?
E_f = predict(np.mean(X_test, axis=0))


def value(X_vector, S, X_sample):
    X_local = copy.deepcopy(X_sample)
    X_local[:, list(S)] = X_vector[list(S)]
    y_actual = predict(X_local)
    return np.mean(y_actual) - E_f

def sample_v1(X_dist, sample_size):
    s_idx    = np.random.randint(low=0, high=len(X_dist), size=sample_size)
    return   X_dist[s_idx]

def sample_v2(X_dist, sample_size):
    return np.array([X_dist[np.random.randint(low=0, high=len(X_dist), size=sample_size),j] for j in N] ).transpose()

def sample_v3(X_dist, dummy):
    return np.mean(X_dist, axis=0).reshape(1,11)
    
def marginal(X_vector, j, S, sample_size):
    X_sample = sample_v3(X_test, sample_size)
    return value(X_vector, S.union(j), X_sample) - value(X_vector, S, X_sample)

def gamma(N,S):
    return math.factorial(len(S)) * math.factorial(len(N) - len(S) - 1) / math.factorial(len(N))

def phi(X_Vector, j, N, sample_size):
    players = N - j
    return np.sum([gamma(N, S) * marginal(X_vector, j, set(S), sample_size) for S in powerset(players)])


In [12]:
X_vector = X_test[1]
phi_T = E_f
phi_i = [phi(X_vector,{player}, N,1000)  for player in N]
phi_i_sorted = sorted(enumerate(phi_i), key=itemgetter(1))
phi_i_sorted.reverse()
for player, p_i in phi_i_sorted:
    print(player_names[player].ljust(20), round(p_i,3))
print('Total phi:    ', round(sum(phi_i),3))
print('Total Reward: ', round(E_f + sum(phi_i),3) )
print('Expected:     ', round(E_f,3))
print('Predicted:    ', round(predict(X_vector),3))
print('Delta:        ', round(predict(X_vector) - (E_f + sum(phi_i)),3)  )


density              0.658
alcohol              0.61
sulphates            0.026
citric_acid          0.011
pH                   0.01
total_sulfur_dioxide 0.003
chlorides            0.0
free_sulfur_dioxide  -0.011
fixed_acidity        -0.065
residual_sugar       -0.246
volatile_acidity     -0.49
Total phi:     0.505
Total Reward:  6.398
Expected:      5.893
Predicted:     6.398
Delta:         0.0


In [13]:
X

array([[ 7.  ,  0.27,  0.36, ...,  3.  ,  0.45,  8.8 ],
       [ 6.3 ,  0.3 ,  0.34, ...,  3.3 ,  0.49,  9.5 ],
       [ 8.1 ,  0.28,  0.4 , ...,  3.26,  0.44, 10.1 ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  2.99,  0.46,  9.4 ],
       [ 5.5 ,  0.29,  0.3 , ...,  3.34,  0.38, 12.8 ],
       [ 6.  ,  0.21,  0.38, ...,  3.26,  0.32, 11.8 ]])